In [ ]:
%pip install langchain langchain_openai langchain_elasticsearch faiss-cpu langchain-community --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/7

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [4]:
raw_text = """
Digital marketing encompasses a broad range of marketing activities that utilize digital channels to connect with customers. At its core, digital marketing aims to reach a targeted audience through various online and electronic means, including social media, email, search engines, and websites. Unlike traditional marketing methods, digital marketing offers unparalleled opportunities for businesses to engage with their audience in real-time, enabling personalized communication and immediate feedback. This real-time interaction not only enhances customer experience but also allows businesses to gather valuable data on consumer behaviors, preferences, and trends, facilitating more effective marketing strategies and campaigns.
The rise of digital marketing can be attributed to the increasing reliance on the internet and digital devices by consumers. As more people spend time online, businesses have shifted their marketing efforts to where their audiences are. Digital marketing leverages this online presence, employing strategies such as search engine optimization (SEO), content marketing, pay-per-click (PPC) advertising, and social media marketing to improve visibility and attract potential customers. These strategies are designed to increase traffic to a company's online platforms, build brand awareness, and ultimately drive conversions and sales. The ability to measure the effectiveness of these strategies through analytics and metrics further underscores the advantage of digital marketing, enabling businesses to refine their approach and maximize return on investment (ROI).
"""

with open("test.txt", "w") as f:
    f.write(raw_text)

In [5]:
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('test.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [6]:
db.similarity_search_with_relevance_scores("digital marketing", k=1)

[(Document(metadata={'source': 'test.txt'}, page_content="Digital marketing encompasses a broad range of marketing activities that utilize digital channels to connect with customers. At its core, digital marketing aims to reach a targeted audience through various online and electronic means, including social media, email, search engines, and websites. Unlike traditional marketing methods, digital marketing offers unparalleled opportunities for businesses to engage with their audience in real-time, enabling personalized communication and immediate feedback. This real-time interaction not only enhances customer experience but also allows businesses to gather valuable data on consumer behaviors, preferences, and trends, facilitating more effective marketing strategies and campaigns.\nThe rise of digital marketing can be attributed to the increasing reliance on the internet and digital devices by consumers. As more people spend time online, businesses have shifted their marketing efforts t

In [7]:
# Adding on extra documents directly within LangChain:
from langchain_core.documents import Document

docs = [Document(page_content='James phoenix worked in digital marketing for 3 years.', metadata={'source': 'James Phoenix'}),
        Document(page_content='Digital marketing is a growing industry.', metadata={'source': 'Wikipedia'}),
        ]

In [8]:
docs[0].metadata

{'source': 'James Phoenix'}

In [9]:
docs[1].metadata

{'source': 'Wikipedia'}

In [10]:
db.add_documents(docs)

['d002856c-fbcf-49c7-ac31-0d9b0d8dc7c5',
 'fcf63706-4035-4006-b007-66ed1f8d460c']

In [13]:
db.similarity_search("James", k=1)

[Document(metadata={'source': 'James Phoenix'}, page_content='James phoenix worked in digital marketing for 3 years.')]

---

## Indexing Becomes Messy Without A Record Manager

Without knowing which documents have been indexed, it is difficult to keep track of the documents that have been indexed. This is especially true when the documents are large and the indexing process is time-consuming.

LangChain solves this with an [Indexing API](https://python.langchain.com/docs/modules/data_connection/indexing) that exposes several different methods to keep track of the documents that have been indexed.

In [ ]:
# WARNING - YOU WILL NEED TO RUN THIS NOTEBOOK LOCALLY WITH DOCKER for ElasticSearch:

`docker run -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" -e "xpack.security.http.ssl.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.9.0`

In [ ]:
from langchain.indexes import SQLRecordManager, index
from langchain_elasticsearch import ElasticsearchStore

collection_name = "test_index"

embedding = OpenAIEmbeddings()

vectorstore = ElasticsearchStore(
    es_url="http://localhost:9200", index_name="test_index", embedding=embedding
)

In [ ]:
namespace = f"elasticsearch/{collection_name}"
record_manager = SQLRecordManager(
    namespace, db_url="sqlite:///record_manager_cache.sql"
)
record_manager.create_schema()

In [ ]:
updated_docs = [
    Document(
        page_content="James phoenix worked in digital marketing for 7 years.",
        metadata={"source": "James Phoenix"},
    ),

]

In [ ]:
def _clear():
    """Hacky helper method to clear content. See the `full` mode section to to understand why it works."""
    index([], record_manager, vectorstore, cleanup="full", source_id_key="source")

In [ ]:
_clear()

In [ ]:
docs

[Document(page_content='James phoenix worked in digital marketing for 3 years.', metadata={'source': 'James Phoenix'}),
 Document(page_content='Digital marketing is a growing industry.', metadata={'source': 'Wikipedia'})]

In [ ]:
# Indexing all of the documents:
index(
    docs,
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 0, 'num_updated': 0, 'num_skipped': 2, 'num_deleted': 0}

In [ ]:
# Updating a single document:
index(
    updated_docs,
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 1}

In [ ]:
# Adding on a new document:
index(
    [Document(page_content="Digital marketing is a growing industry.", metadata={"source": "Wikipedia - Digital Marketing"})],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [ ]:
# Skipping documents because the document hash is exactly the same:
index(
    [Document(page_content="Digital marketing is a growing industry.", metadata={"source": "Wikipedia - Digital Marketing"})],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 0, 'num_updated': 0, 'num_skipped': 1, 'num_deleted': 0}

## There are 3 modes of using the Index API.


| Cleanup Mode | De-Duplicates Content | Parallelizable | Cleans Up Deleted Source Docs | Cleans Up Mutations of Source Docs and/or Derived Docs | Clean Up Timing      |
|--------------|-----------------------|----------------|-------------------------------|--------------------------------------------------------|----------------------|
| None         | ✅                     | ✅              | ❌                             | ❌                                                      | -                    |
| Incremental  | ✅                     | ✅              | ❌                             | ✅                                                      | Continuously         |
| Full         | ✅                     | ❌              | ✅                             | ✅                                                      | At end of indexing   |


Read more here: https://python.langchain.com/docs/modules/data_connection/indexing